
## Import 

In [ ]:
import os, sys, pickle

import requests

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
import seaborn as sns
sns.set()
import plotly.express as px

from sklearn.model_selection import * 
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.impute import * 
from sklearn.preprocessing import * 
from sklearn.ensemble import * 

import shap

## Load data

In [ ]:
url = "https://gist.githubusercontent.com/AlexandreGazagnes/9018022652ba0933dd39c9df8a600292/raw/0845ef4c2df4806bb05c8c7423dc75d93e37400f/titanic_train_raw_csv"

In [ ]:
df = pd.read_csv (url)

In [ ]:
df.head()

## Select 

In [ ]:
cols = ["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
df = df.loc[:, cols]
df.head()

In [ ]:
df.info()

## Prepare

In [ ]:
X = df.drop(columns="Survived")
y = df.Survived

In [ ]:
X_train, X_test, y_train, y_test  = train_test_split(X, y)

## Model

In [ ]:

categorical_features = ["Embarked", "Sex"]
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_features),
    ],
    remainder='passthrough'
)

In [ ]:
pipe = Pipeline([
    ("preprocessor", preprocessor),
    ("imputer",      KNNImputer() ),
    ("scaler",       "passthrough"),
    ("estimator",    RandomForestClassifier()),
    ])

In [ ]:
grid = GridSearchCV(pipe, {}, cv=10, return_train_score=True, n_jobs=-1)

## Train

In [ ]:
grid.fit(X_train, y_train)

## Test

In [ ]:
res = pd.DataFrame(grid.cv_results_)
res = res.loc[:, [i for i in res.columns if "split" not in i ]]
res

In [ ]:
grid.score(X_test, y_test)

In [ ]:
model = grid.best_estimator_
model

## Explain

In [ ]:
random = model['estimator']

In [ ]:
px.bar(random.feature_importances_)

In [ ]:
explainer = shap.TreeExplainer(model['estimator'])


In [ ]:
explainer.feature_

In [ ]:
def prepare(_x) : 
    """ """

    _x = model["preprocessor"].transform(_x)
    _x = model["imputer"].transform(_x)
    # _x = model["scaler"].transform(_x)
    
    return _x
    
shap_values = explainer.shap_values(prepare(X_test))

In [ ]:
shap.summary_plot(shap_values, prepare(X_test), plot_type="bar")

## Save

In [ ]:
with open("./models/model.pk", "wb") as f : 
    pk = pickle.dumps(grid)
    f.write(pk)